In [34]:
import configparser
import numpy as np
import time
import xgboost as xgb
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pycm import ConfusionMatrix

from data_loader import DataLoader
from feature_selector import BaselineSelector, AnalysisSelector
from utils import balanced_split, parse_config_features, shape_wrapper, plot_confusion_matrix
from imblearn.over_sampling import RandomOverSampler, SMOTE


In [2]:
seed = 420

def get_model(config: configparser.ConfigParser):
    """
    Returns model by name
    """
    hyperparams = config['Model']
    name = hyperparams['model_name']
    random_state = None if hyperparams['random_state'] == 'None' else int(hyperparams['random_state'])

    if name == 'logistic_regression':
        return LogisticRegression(random_state=random_state)
    
    elif name == 'knn':
        n_neighbors = int(hyperparams['n_neighbors'])
        return KNeighborsClassifier(n_neighbors=n_neighbors, random_state=random_state)
    
    elif name == 'svm':
        return SVC(random_state=random_state)
    
    elif name == 'random_forest':
        n_estimators = int(hyperparams['n_estimators'])
        max_depth = int(hyperparams['max_depth'])
        return RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    
    elif name == 'xgboost':
        n_estimators = int(hyperparams['n_estimators'])
        max_depth = int(hyperparams['max_depth'])
        eta = float(hyperparams['eta'])
        return xgb.XGBClassifier(learning_rate = 0.2, n_estimators=n_estimators, objective='multi:softmax', num_class=3, max_depth=max_depth, random_state=random_state)
    
    elif name == 'naive_bayes':
        return GaussianNB(**hyperparams)
    
    else:
        raise ValueError(f'Unknown model name: {name}')

In [3]:
def get_metrics(y_true, y_pred):
    """
    Returns metrics for classification
    """
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred, average='macro'),
        'precision': precision_score(y_true, y_pred, average='macro'),
        'recall': recall_score(y_true, y_pred, average='macro'),
        'confusion_matrix': ConfusionMatrix(y_true, y_pred)
    }

In [ ]:
def train_model(config: configparser.ConfigParser):
    """
    Trains model and returns metrics
    """
    X = pd.read_csv(config['Data']['features_path'])

    y = np.loadtxt(config['Data']['labels_path'], dtype=np.int32)

In [36]:
config_path = './config.ini'
config = configparser.ConfigParser()
config.read(config_path)
config['Model']['model_name']

'svm'

In [22]:
X = pd.read_csv(config['Data']['features_path'])

y = np.load(config['Data']['labels_path'])
par_ids = np.load(config['Data']['par_ids_path'])
X['par_ids'] = par_ids

In [20]:
n_folds = int(config['Train']['n_folds'])
num_test_part = int(config['Train']['num_test_part'])

In [38]:
dataset_path = config['Data']['dataset_path']
for k in range(n_folds):
    
    train_ids, test_ids = balanced_split(dataset_path, seed = seed)

    X_train = X[X['par_ids'].isin(train_ids)]
    X_test = X[X['par_ids'].isin(test_ids)]

    y_train = y[np.isin(par_ids, train_ids)]
    y_test = y[np.isin(par_ids, test_ids)]

    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)

    X_train = X_train.drop(['par_ids'], axis=1)
    X_test = X_test.drop(['par_ids'], axis=1)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # oversample
    oversample = SMOTE()
    X_train, y_train = oversample.fit_resample(X_train, y_train)

    model = get_model(config)
    
    print('starting training')
    # train and evaluate model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('finished training')
    metrics = get_metrics(y_test, y_pred)
    print(metrics)

starting training
